# Lesson 12: Image & File Generation

🔴 **Advanced** · ⏱ **25 min**

---

lyzr-adk agents can generate images using DALL-E 3 or Gemini's image models, and produce file artifacts like CSVs, reports, and code files. This lesson shows how to configure image models and prompt your agent for visual and file outputs.

## What you'll learn

- Configure DALL-E 3 and Gemini image models on an agent using `set_image_model()`
- Prompt an agent to generate images and interpret the response format
- Understand how image URLs and artifact references are returned
- Use agents to generate file artifacts like CSVs and structured reports

## Prerequisites

> **Note:** This is an **optional advanced lesson**. It is not required to complete the core lyzr-adk series.

Before starting this lesson, you should have completed (or be familiar with):

- **Lesson 1** — Getting Started (agent creation basics)
- **Lesson 2** — Providers and Models
- **Lesson 3** — Agent Lifecycle
- **Lesson 4** — Structured Outputs

You will also need:
- A valid `LYZR_API_KEY` set as an environment variable (or replace the placeholder in the setup cell)
- An **OpenAI API key** (for DALL-E 3) or a **Google API key** (for Gemini image models) linked to your Lyzr account, depending on which image model you choose

In [ ]:
!pip install lyzr-adk -q

In [ ]:
import os
from lyzr import Studio

API_KEY = os.getenv("LYZR_API_KEY", "YOUR_LYZR_API_KEY")
studio = Studio(api_key=API_KEY)
print("Ready!")

## Image Generation in lyzr-adk

By default, a lyzr-adk agent only generates text. To enable image generation, you call `set_image_model()` on the agent after creating it. This tells the agent which image backend to use when it encounters a request to generate an image.

Three image models are supported:

| Model | String | Provider | Best for |
|-------|--------|----------|---------|
| DALL-E 3 | `"dalle-3"` | OpenAI | Photo-realistic, detailed |
| Gemini Flash | `"gemini-flash"` | Google | Fast generation |
| Gemini Pro | `"gemini-pro"` | Google | High quality |

The `set_image_model()` call can be made at any time before you ask the agent to generate an image. You can also switch models between calls by calling it again with a different string.

```python
agent.set_image_model("dalle-3")       # DALL-E 3 via OpenAI
agent.set_image_model("gemini-flash")  # Gemini Flash image gen
agent.set_image_model("gemini-pro")    # Gemini Pro image gen
```

## Setting Up an Image Generation Agent with DALL-E 3

Create an agent as normal, then call `set_image_model()` to enable image generation. The underlying LLM (the `provider`) handles reasoning and prompt interpretation; the image model handles the actual image synthesis.

In [ ]:
# Create an agent and configure it with DALL-E 3
image_agent = studio.create_agent(
    name="Visual Creator",
    provider="openai/gpt-4o",
    role="Creative visual designer",
    goal="Generate high-quality images based on user descriptions",
    instructions=(
        "When asked to create or generate an image, produce a detailed, vivid image. "
        "Always confirm what image you're generating before creating it."
    )
)

# Set the image model
image_agent.set_image_model("dalle-3")
print(f"✅ Agent configured with DALL-E 3")
print(f"   Agent ID: {image_agent.id}")

## Generating Your First Image

Call `agent.run()` with a descriptive image prompt. The response object works the same as a text response — check `response.response` for the agent's message, and also inspect `response.image_url` or `response.artifact` for the generated image reference.

The exact attribute name depends on the SDK version, so the cell below checks both possibilities and falls back to inspecting all available attributes.

In [ ]:
# Generate an image
response = image_agent.run(
    "Generate an image of a cozy coffee shop on a rainy evening, "
    "with warm lighting, wooden tables, and people reading books."
)

print(f"Response: {response.response}")

# The response may contain an image URL or artifact reference
# Depending on the SDK version, check response attributes
if hasattr(response, 'image_url') and response.image_url:
    print(f"\nImage URL: {response.image_url}")
elif hasattr(response, 'artifact') and response.artifact:
    print(f"\nArtifact: {response.artifact}")
else:
    print("\n(Check response object for image data)")
    print(f"Response type: {type(response)}")
    print(f"Response attributes: {[a for a in dir(response) if not a.startswith('_')]}")

## Switching to Gemini Image Models

You are not locked into one image model per agent. Call `set_image_model()` again at any time to switch backends. You can also create separate agents for each model to run them in parallel.

Use **Gemini Flash** when generation speed is the priority, and **Gemini Pro** when you need higher output quality.

In [ ]:
# Switch to Gemini Flash for faster generation
gemini_agent = studio.create_agent(
    name="Gemini Visual Creator",
    provider="google/gemini-2.0-flash",
    role="Creative visual designer",
    goal="Generate images quickly using Gemini",
    instructions="Generate creative, high-quality images based on descriptions."
)

gemini_agent.set_image_model("gemini-flash")
print("✅ Agent configured with Gemini Flash")

response2 = gemini_agent.run(
    "Create an image of a robot learning to paint in an art studio, "
    "surrounded by canvases and with paint on its hands."
)
print(f"Response: {response2.response}")

## File Generation

Beyond images, agents can produce structured file outputs — CSVs, Markdown reports, JSON data, code files, and more. File generation does not require a special model configuration like `set_image_model()`. Instead, you prompt the agent to create the file content, and the response is returned as a text artifact that you can write to disk or process programmatically.

Good system instructions for file-generating agents should:
- Specify the expected output format (CSV, JSON, Markdown, etc.)
- Indicate how many rows or sections to include
- Ask for realistic or representative data rather than placeholder values

The agent uses its LLM to reason about the request and produce well-formed file contents, which are returned in `response.response` or a dedicated `response.artifact` field.

In [ ]:
# Create an agent for file generation
file_agent = studio.create_agent(
    name="Report Generator",
    provider="openai/gpt-4o",
    role="Data analyst and report writer",
    goal="Generate structured data files and reports",
    instructions=(
        "When asked to create a file or report, generate complete, realistic data. "
        "For CSV files, include headers and at least 5-10 rows of data. "
        "For reports, be concise and use clear sections."
    )
)
print(f"✅ File generation agent created: {file_agent.id}")

In [ ]:
# Ask agent to generate a CSV report
response3 = file_agent.run(
    "Generate a CSV file of Q1 2026 sales data with columns: "
    "Date, Product, Region, Units_Sold, Revenue_USD. Include 10 rows."
)

print(f"Response: {response3.response}")

# Check for file artifact
if hasattr(response3, 'artifact') and response3.artifact:
    print(f"\nFile artifact: {response3.artifact}")

## Prompting Tips for Better Image Generation

Image generation models respond strongly to the specificity and structure of the prompt. Vague prompts produce generic results; detailed prompts with clear style, mood, and composition guidance produce much better outputs.

A well-structured image prompt typically includes:

| Element | Example |
|---------|----------|
| **Subject** | "a golden retriever dog" |
| **Setting** | "in a sunlit park" |
| **Style** | "watercolor painting" |
| **Lighting** | "soft morning light" |
| **Mood** | "peaceful and warm" |
| **Perspective** | "close-up portrait" |
| **Quality hint** | "highly detailed, professional illustration" |

You do not need all elements in every prompt, but including at least 3-4 of them consistently improves results.

In [ ]:
# Prompt engineering for image quality
prompts = [
    # Vague (less effective)
    "A city",

    # Specific (more effective)
    "A futuristic Tokyo skyline at dusk, cyberpunk aesthetic, "
    "neon lights reflecting on wet streets, highly detailed, cinematic lighting",

    # With style guidance
    "Portrait of a golden retriever dog, watercolor painting style, "
    "soft pastel colors, white background, professional illustration"
]

print("Prompt engineering examples:\n")
for i, prompt in enumerate(prompts, 1):
    print(f"Prompt {i}: {prompt[:80]}...")
    print()

print("✅ More specific prompts = better image results")
print("   Include: subject, setting, style, lighting, mood, perspective")

## Common Mistake: Not Calling `set_image_model()` Before Asking for Images

If you ask an agent to generate an image without first calling `set_image_model()`, the agent will not have an image backend configured. Instead of synthesizing an image, it will fall back to describing the image in text — which is not the same thing.

**Rule:** Always call `agent.set_image_model("dalle-3")` (or another supported model string) before issuing image generation prompts.

In [ ]:
# Without set_image_model, agent will describe the image rather than generate it
no_image_agent = studio.create_agent(
    name="No Image Config",
    provider="openai/gpt-4o",
    role="Assistant",
    goal="Help",
    instructions="Help users."
)
# set_image_model() is intentionally NOT called here
response4 = no_image_agent.run("Generate an image of a sunset over the ocean.")
print(f"Without image model configured:\n{response4.response[:200]}")
print("\n(Agent will describe, not generate, without set_image_model)")

print("\n✅ Always call agent.set_image_model('dalle-3') before asking for images.")

## Exercise: Generate and Compare Two Images

Put it all together. Your task is to:

1. Create an image generation agent with a model of your choice
2. Configure it with `set_image_model()`
3. Generate **Image 1** — a realistic scene with specific details
4. Generate **Image 2** — the same subject but with an explicit artistic style (watercolor, oil painting, sketch, etc.)
5. Print both prompts side by side to compare your approach

Fill in the `TODO` sections in the cell below.

In [ ]:
# TODO: Create an image generation agent
my_image_agent = studio.create_agent(
    name=...,
    provider=...,  # openai/gpt-4o for DALL-E or google/gemini-2.0-flash for Gemini
    role=...,
    goal=...,
    instructions=...
)

# TODO: Configure the image model
my_image_agent.set_image_model(...)  # "dalle-3", "gemini-flash", or "gemini-pro"

# TODO: Generate Image 1 — a realistic scene
prompt_1 = "..."  # Describe a realistic scene with specific details
response_1 = my_image_agent.run(prompt_1)
print(f"Image 1 response: {response_1.response[:200]}")

# TODO: Generate Image 2 — an artistic/stylized scene
prompt_2 = "..."  # Same subject but with an artistic style (watercolor, oil painting, etc.)
response_2 = my_image_agent.run(prompt_2)
print(f"\nImage 2 response: {response_2.response[:200]}")

# TODO: Print both prompts side by side for comparison
print(f"\nPrompt 1: {prompt_1}")
print(f"Prompt 2: {prompt_2}")

## Summary

### Image models at a glance

| Model | String | Provider | Strength |
|-------|--------|----------|---------|
| DALL-E 3 | `"dalle-3"` | OpenAI | Photo-realistic, fine detail |
| Gemini Flash | `"gemini-flash"` | Google | Fastest generation |
| Gemini Pro | `"gemini-pro"` | Google | Highest quality from Google |

### File generation capabilities

| File type | How to prompt |
|-----------|---------------|
| CSV | "Generate a CSV with columns X, Y, Z and 10 rows of data" |
| Markdown report | "Write a report with sections: Overview, Data, Conclusion" |
| JSON | "Return a JSON object with fields A, B, C" |
| Code file | "Write a Python script that does X" |

### Prompt engineering tips

- Include subject, setting, style, lighting, mood, and perspective
- More specific = better results; vague prompts produce generic output
- Name an artistic style explicitly ("watercolor", "oil painting", "cinematic") to guide aesthetics
- For file generation, specify the schema, row count, and data realism in your prompt

### Key takeaways

- `agent.set_image_model("dalle-3")` must be called before asking for images — without it, the agent describes instead of generating
- The same agent can switch image models at any time by calling `set_image_model()` again
- Image output appears in `response.image_url` or `response.artifact` depending on the SDK version
- File artifacts are returned in `response.response` or `response.artifact` and can be written to disk directly
- Image generation and file generation are compatible with memory, sessions, and tools

## Next Steps

You have completed Lesson 12. From here you can:

- **Lesson 13: Advanced Features** — Reflection, Bias Check, and Groundedness for production-grade agent quality
- **Back to Lesson 10: Capstone Project** — if you haven't completed it yet, the capstone brings together memory, tools, RAI, and knowledge bases into a full end-to-end build
- **Back to Lesson 11: Streaming** — real-time token-by-token output for chat interfaces and CLIs

---

| Lesson | Topic |
|--------|-------|
| [01](./01_getting_started.ipynb) | Getting Started |
| [02](./02_providers_and_models.ipynb) | Providers and Models |
| [03](./03_agent_lifecycle.ipynb) | Agent Lifecycle |
| [04](./04_structured_outputs.ipynb) | Structured Outputs |
| [05](./05_memory_and_sessions.ipynb) | Memory and Sessions |
| [06](./06_tools_and_functions.ipynb) | Tools and Functions |
| [07](./07_knowledge_bases_rag.ipynb) | Knowledge Bases (RAG) |
| [08](./08_contexts.ipynb) | Contexts |
| [09](./09_rai_guardrails.ipynb) | RAI Guardrails |
| [10](./10_capstone_project.ipynb) | Capstone Project |
| [11](./11_streaming.ipynb) | Streaming |
| **12** | **Image & File Generation (this lesson)** |